In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
import struct

In [2]:
f = open('MNIST/train-images.idx3-ubyte', 'rb')
magic, size, rows, cols = struct.unpack('>IIII', f.read(16))
print(magic, size, rows, cols)
imgdata = f.read()
imgdata = np.frombuffer(imgdata, dtype=np.uint8)
imgdata = imgdata.reshape(size, rows*cols)

2051 60000 28 28


In [3]:
def showimg(data):
    img1 = data.astype(np.float32)
    img1 = img1.reshape(rows, cols)
    img1 = np.asarray(img1)
    plt.imshow(img1, cmap='gray')
    plt.show()

In [4]:
l = open('MNIST/train-labels.idx1-ubyte', 'rb')
magic, size = struct.unpack(">II", l.read(8))
print(magic, size)
labeldata = l.read()
labeldata = np.frombuffer(labeldata, dtype=np.uint8).astype(np.int32)

2049 60000


In [5]:
def init_layers():
    np.random.seed(1)

    W1 = np.random.rand(32, 784) - 0.5
    B1 = np.random.rand(32, 1) - 0.5

    W2 = np.random.rand(16, 32) - 0.5
    B2 = np.random.rand(16, 1) - 0.5

    W3 = np.random.rand(10, 16) - 0.5
    B3 = np.random.rand(10, 1) - 0.5

    W = [None, W1, W2, W3]
    B = [None, B1, B2, B3]

    return W, B


def ReLU(X):
    return np.maximum(X, 0)

def ReLU_deriv(Z):
    return Z > 0

def softmax(X):
    return np.exp(X) / sum(np.exp(X))

# derivative of softmax at output layer is a(L) - y



def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    sig = sigmoid(x)
    return sig * (1 - sig)




def one_hot_all_y(labels):
    Y = np.zeros((10, len(labels)))
    for i in range(len(labels)):
        Y[labels[i]][i] = 1
    return Y

# def one_hot(Y):
#     one_hot_Y = np.zeros((Y.size, Y.max() + 1))
#     one_hot_Y[np.arange(Y.size), Y] = 1
#     one_hot_Y = one_hot_Y.T
#     return one_hot_Y

def forward_prop(W, X, B):
    A0 = X

    Z1 = W[1].dot(X) + B[1]
    A1 = ReLU(Z1)

    Z2 = W[2].dot(A1) + B[2]
    A2 = ReLU(Z2)

    Z3 = W[3].dot(A2) + B[3]
    A3 = sigmoid(Z3)
    Z = [None, Z1, Z2, Z3]
    A = [A0, A1, A2, A3]

    return Z, A 

def backward_prop(A, Z, W, Y):
    Y_oh = one_hot_all_y(Y)
    m = A[0].shape[1]

    # dZ3 = (A[3] - Y_oh) softmax
    dZ3 = 2 * (A[3] - Y_oh) * 1
    dW3 = 1 / m * dZ3.dot(A[2].T)
    dB3 = 1 / m * np.sum(dZ3)


    dZ2 = W[3].T.dot(dZ3) * ReLU_deriv(Z[2])
    dW2 = 1 / m * dZ2.dot(A[1].T)
    dB2 = 1 / m * np.sum(dZ2)

    dZ1 = W[2].T.dot(dZ2) * ReLU_deriv(Z[1])
    dW1 = 1 / m * dZ1.dot(A[0].T)
    dB1 = 1 / m * np.sum(dZ1)
    
    dW = [None, dW1, dW2, dW3]
    dB = [None, dB1, dB2, dB3]


    return dW, dB

def update_wb(W, B, dW, dB, alpha=0.1):
    for i in range(1, len(W) ):
        W[i] = W[i] - alpha * dW[i]
        B[i] = B[i] - alpha * dB[i]
    
    return W, B

In [6]:
def get_predictions(A):
    return np.argmax(A, 0)

def get_accuracy(predictions, Y):
    print(predictions[:10], Y[:10])
    return np.sum(predictions == Y) / Y.size


def gradient_descent(X, Y, alpha=0.1, iterations=100):
    W, B = init_layers()
    for i in range(iterations):
        Z, A = forward_prop(W, X, B)
        dW, dB = backward_prop(A, Z, W, Y)
        W, B = update_wb(W, B, dW, dB, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A[3])
            print(get_accuracy(predictions, Y))
    return W, B

In [7]:
X = imgdata.T.copy()
Y = labeldata.copy()

test_percent = 0.2
test_idx = int(X.shape[1] * test_percent)
X_test, X_train = X[:,:test_idx], X[:,test_idx:]
X_test = X_test / 255.
X_train = X_train / 255.
Y_test, Y_train = Y[:test_idx], Y[test_idx:]

W, B = gradient_descent(X_train, Y_train, 0.1, 500)

Iteration:  0
[2 0 2 2 2 2 2 8 0 2] [7 4 6 5 0 6 9 7 0 8]
0.1025
Iteration:  10
[2 3 3 3 5 3 3 3 3 7] [7 4 6 5 0 6 9 7 0 8]
0.11025
Iteration:  20
[2 3 3 1 2 3 3 2 3 1] [7 4 6 5 0 6 9 7 0 8]
0.18002083333333332
Iteration:  30
[2 9 3 1 2 3 9 2 9 1] [7 4 6 5 0 6 9 7 0 8]
0.2747291666666667
Iteration:  40
[7 9 3 1 0 1 9 2 0 1] [7 4 6 5 0 6 9 7 0 8]
0.4149375
Iteration:  50
[7 4 3 1 0 8 4 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.4934375
Iteration:  60
[7 4 3 1 0 8 6 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.5533333333333333
Iteration:  70
[7 4 6 1 0 8 6 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.6127291666666667
Iteration:  80
[7 4 6 1 0 8 6 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.6582083333333333
Iteration:  90
[7 4 6 1 0 8 6 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.6965
Iteration:  100
[7 4 6 2 0 8 6 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.7270208333333333
Iteration:  110
[7 4 6 2 0 6 9 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.7532291666666666
Iteration:  120
[7 4 6 2 0 6 9 7 0 8] [7 4 6 5 0 6 9 7 0 8]
0.7745
Iteration:  130
[7 4 6 2 0 6 9 7 0 8] [7 4 6 5 0

In [8]:
def test_accuracy(W, B, X, Y):
    Z, A = forward_prop(W, X, B)
    predictions = get_predictions(A[3])
    print(get_accuracy(predictions, Y))

test_accuracy(W, B, X_test, Y_test)

[5 0 4 1 9 2 1 3 1 4] [5 0 4 1 9 2 1 3 1 4]
0.9129166666666667


In [14]:
with open("digits.wab", "wb") as buffer:
    # num of layers
    buffer.write(struct.pack('i', 4))
    # nodes per layer 
    buffer.write(struct.pack("iiii", 784, 32, 16, 10))
    for i in range(1, len(W)):
        buffer.write(W[i].astype(np.float32).tobytes())
        buffer.write(B[i].astype(np.float32).tobytes())
    
    


In [15]:
print(W[1])

[[-0.082978    0.22032449 -0.49988563 ...  0.47522234 -0.17968067
   0.48249094]
 [ 0.13630613 -0.12490898  0.3574845  ... -0.32150517  0.46987057
   0.24193953]
 [-0.24020831  0.08123534  0.45526225 ...  0.1982818  -0.3533913
   0.11246113]
 ...
 [ 0.10878973 -0.46599038 -0.30684345 ... -0.44884619  0.33541706
   0.26660803]
 [-0.41782107  0.19786519  0.47512939 ...  0.15942875  0.23343124
  -0.21456471]
 [-0.14139756  0.092913   -0.45361685 ...  0.44740965 -0.48401982
   0.36748756]]


In [16]:
print(B[3])

[[-1.52805321]
 [-1.32176467]
 [-1.28267291]
 [-0.82654748]
 [-1.63870584]
 [-1.39853099]
 [-1.53373373]
 [-1.32263423]
 [-1.73717508]
 [-1.30515405]]


In [24]:
print("{ ")
for val in X_test[:,0]:
    print(f"{val}f,")
print("}")

{ 
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.0f,
0.011764705882352941f,
0.07058823529411765f,
0.07058823529411765f,
0.0705882352941176

In [25]:
X_test.shape

(784, 12000)